In [1]:
from prefect import task
import requests
import pandas as pd
import numpy as np
import sqlalchemy
from decouple import config 
import psycopg2
from datetime import datetime

In [2]:
host = config('AWS_DATABASE_URL')
db = sqlalchemy.create_engine(host)
conn = db.connect()
curs = conn.connection.cursor()

In [ ]:
conn = db.connect()
curs = conn.connection.cursor()
curs.execute(""" 
            CREATE TABLE "comments"(
            by VARCHAR,
            id INT,
            text TEXT,
            time TIMESTAMP,
            sentiment FLOAT
            )
            """)
conn.close()
curs.close()

In [3]:
conn = db.connect()
curs = conn.connection.cursor()
curs.execute(""" 
            CREATE TABLE "comments"(
            by VARCHAR,
            id INT,
            text TEXT,
            time TIMESTAMP,
            sentiment FLOAT
            )
            """)
conn.close()
curs.close()

DuplicateTable: relation "comments" already exists


In [7]:
def hn_scrape(i,comments_list):
    r = requests.get('https://hacker-news.firebaseio.com/v0/item/'+str(i)+'.json').json()
    try:
        if ('deleted' in r.keys()):
            pass
        else:
            if r["type"] == 'comment':
                t = (r["by"],r["id"],r["text"],r["time"])
                comments_list.append(t)
    except:
        pass

def sentiment(text):
    payload = {'text':text}
    try:
        return requests.get('https://crawftv-nlp-api.herokuapp.com/sentiment',params=payload).json()["compound"]
    except:
        pass
    
def get_date(ts):
            return datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d")
    
@task
def extract():
    comments_list = []
    max_item = requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty").json()
    for i in range(max_item-10,max_item):
        hn_scrape(i,comments_list)
    return comments_list
    
    

@task
def transform(comments_list):
    comments= pd.DataFrame(columns = ["by", "id","text","time"], data=comments_list)
    comments["text"] = comments["text"].str.replace("&quot;","")
    comments["text"] = comments["text"].str.replace("&#x27;","'")
    comments["text"] = comments["text"].str.replace("&gt; "," ")
    comments["text"] = comments["text"].str.replace("<p>"," ")
    comments["text"] = comments["text"].str.replace("<a>"," ")
    comments["text"] = comments["text"].str.replace("</a>"," ")
    comments["text"] = comments["text"].str.replace("<i>"," ")
    comments["text"] = comments["text"].str.replace("</i>"," ")
    comments["text"] = comments["text"].str.replace("&#x2F;",'')
    comments["text"] = comments["text"].str.replace("https:",' ')
    comments["text"] = comments["text"].str.replace("\\n",' ')
    comments["time"] = comments["time"].apply(get_date)
    comments = comments.dropna()
    print(comments)
    return comments


@task
def load(comments):
    comments["sentiment"] = comments["text"].apply(sentiment)
    comments.to_sql(name="comments",con=db, if_exists="append",chunksize=500)



In [14]:
from prefect import Flow


max_item = requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty").json()


with Flow('ETL') as flow:
    e = extract()
    t = transform(e)
    l = load(t)


flow.run()
conn.close()
curs.close()

[2019-09-11 20:19:23,435] INFO - prefect.FlowRunner | Beginning Flow run for 'ETL'
[2019-09-11 20:19:23,435] INFO - prefect.FlowRunner | Starting flow run.
[2019-09-11 20:19:23,438] INFO - prefect.TaskRunner | Task 'extract': Starting task run...
[2019-09-11 20:19:24,546] INFO - prefect.TaskRunner | Task 'extract': finished task run for task with final state: 'Success'
[2019-09-11 20:19:24,547] INFO - prefect.TaskRunner | Task 'transform': Starting task run...
[2019-09-11 20:19:24,556] INFO - prefect.TaskRunner | Task 'transform': finished task run for task with final state: 'Success'
[2019-09-11 20:19:24,557] INFO - prefect.TaskRunner | Task 'load': Starting task run...


               by        id  \
0       wilsonnb3  20944204   
1     dragontamer  20944205   
2         ekianjo  20944206   
3        zeristor  20944207   
4   mullingitover  20944208   
5  blackflame7000  20944209   
6           scotu  20944210   
7      alistairSH  20944212   
8     Supermancho  20944213   

                                                text        time  
0  Talkbass is still the best place on the web to...  2019-09-11  
1               Strangely enough, it works with -O2.  2019-09-11  
2  What is incredibly difficult about being a jou...  2019-09-11  
3  Isn’t British mass market chocolate a laughabl...  2019-09-11  
4  It has marked effects on young adults, and red...  2019-09-11  
5  It's much easier to create additional emails t...  2019-09-11  
6  as often happens, this focuses on technical as...  2019-09-11  
7  But phone numbers are not unique.  They are re...  2019-09-11  
8  If it's chemical propulsion, you're dead when ...  2019-09-11  


[2019-09-11 20:19:35,049] INFO - prefect.TaskRunner | Task 'load': finished task run for task with final state: 'Success'
[2019-09-11 20:19:35,050] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


## test

In [ ]:
comments

In [4]:
conn = db.connect()
curs = conn.connection.cursor()
curs.execute(""" SELECT COUNT(id) FROM "comments" """)
table_max = curs.fetchall()
print(table_max)
conn.close()
curs.close()

[(9,)]
